In [1]:
import math

In [2]:
class LCG:
    def __init__(self, seed, a=561860773102413563, c=0, m=2**60-93):
        self.seed = seed
        self.a = a
        self.c = c
        self.m = m
        self.state = seed

    def next(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / self.m  # Normalize to [0, 1)
    
    def next_in_range(self, a, b):
        return a + (b - a) * self.next()

In [3]:
class MonteCarlo:
    def __init__(self, N, PRNG_object):
        self.N = N
        self.PRNG = PRNG_object
    
    def integrate(self, f, a, b):
        mult = (b - a) / self.N
        
        generatedValues = []
        for _ in range(self.N):
            randomArg = self.PRNG.next_in_range(a, b)
            randomFuncVal = f(randomArg)

            generatedValues.append(randomFuncVal)
        
        return mult * sum(generatedValues)

In [4]:
class CDM:
    def __init__(self, h):
        self.h = h
    
    def diff(self, f, x):
        numerator = f(x + self.h) - f(x - self.h)
        denominator = 2 * self.h

        return numerator / denominator

In [5]:
class Newton:
    def __init__(self, f, CDM_object, tol=1e-6, max_iter=1000):
        self.f = f
        self.CDM = CDM_object
        self.tol = tol 
        self.max_iter = max_iter

    def solve(self, y, x0):
        x = x0
        for _ in range(self.max_iter):
            f_x = self.f(x) - y
            f_prime_x = self.CDM.diff(self.f, x)
            if abs(f_prime_x) < 1e-10:
                raise ValueError("Derivative is zero, method fails.")
            x_new = x - f_x / f_prime_x
            if abs(x_new - x) < self.tol:
                return x_new
            x = x_new
        # return x_new
        raise ValueError(f"Method did not converge.({x_new})")

In [6]:
if __name__ == '__main__':
    lcg        = LCG(seed=42)
    monteCarlo = MonteCarlo(1000000, lcg)
    
    def pdf(x): # f_X
        return 1 / (math.sqrt(0.4 * math.pi) * x) \
            * math.exp(-(math.log(x) - 2)**2 / 0.4)

    def subs(u):
        return pdf(math.tan(u)) * (1 / math.cos(u)**2) * math.tan(u)

    def cdf(x): # F_X
        return monteCarlo.integrate(pdf, 0, x)

    print(monteCarlo.integrate(subs, 0, math.pi/2))

    cdm = CDM(h=1e-6)
    newton = Newton(cdf, cdm, 0.001, 100)

    def inverse(y, x0): # x = f^-1(y)
        return newton.solve(y, x0)

8.190621308758706


In [7]:
def foo(x):
    return 1 - (50/x)**(25)

def inv_foo(y):
    return 50 / ((1 - y)**(1/25))

In [8]:
if __name__ == '__main__':
    lcg        = LCG(seed=42)
    monteCarlo = MonteCarlo(100000, lcg)
    
    def pdf(x): # f_X
        return (1 / 2) * (50 / x)**(26)

    def cdf(x): # F_X
        return monteCarlo.integrate(pdf, 50, x)

    # for i in range(51, 60):
    #     print(f'{foo(i)}   {cdf(i)}')

    cdm    = CDM(h=1e-6)
    newton = Newton(cdf, cdm, tol=1e-6, max_iter=1000)

    def inverse(y, x0): # x = f^-1(y)
        return newton.solve(y, x0)

    # print(f'-'*200)

    # print(inv_foo(0.355))
    # print(inverse(0.355, 50))

In [9]:
import scipy.special
from scipy.stats import lognorm
import numpy as np

In [10]:
mu = 2
sigma = math.sqrt(0.2)

lognorm_dist = lognorm(s=sigma, scale=np.exp(mu))

def foo(x):
    # return math.sin(x)
    # return x**3 - 2 * x - 5
    return lognorm_dist.cdf(x)

def inv_foo(y):
    return lognorm_dist.ppf(y)


In [ ]:
if __name__ == '__main__':
    lcg        = LCG(seed=42)
    monteCarlo = MonteCarlo(100000, lcg)
    
    def pdf(x): # f_X
        # return 1 / (math.sqrt(0.4 * math.pi) * x) \
        #     * math.exp(-(math.log(x) - 2)**2 / 0.4)
        return (1 / 2) * (50 / x)**(26)

    def erf(x): # erf(x)
        def integrand(t):
            return np.exp(-t**2)

        integral_value = monteCarlo.integrate(integrand, 0, x)
        return (2 / np.sqrt(np.pi)) * integral_value

    def cdf(x): # F_X
        # return monteCarlo.integrate(pdf, 0, x)
        # return monteCarlo.integrate(pdf, 50, x)
        # return 1/2 + 1/2 * scipy.special.erf((np.log(x) - 2)/(np.sqrt(0.4)))
         return 1/2 + 1/2 * erf((np.log(x) - 2)/(np.sqrt(0.4)))

    # for i in range(0, 15):
    #     print(f'{foo(i)}   {cdf(i)}')

    cdm    = CDM(h=1e-6)
    newton = Newton(cdf, cdm, tol=1e-6, max_iter=1000)

    def inverse(y, x0): # x = f^-1(y)
        return newton.solve(y, x0)

    # print(f'foo: {foo(3.1519)}')
    # print(f'cdf(51): {cdf(51)}')

    print(f'')

    data = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    guesses = [0, 3, 6, 9, 12, 15]

    for el in data:
        print(f'scipy: {inv_foo(el)}')
        for guess in guesses:
            try: 
                print(f'mine: {newton.solve(el, guess)}')
                break
            except:
                print(f'guess {guess} did not converge') 
        print('-'*200)

    # print(inv_foo(0.5))
    # print(newton.solve(0.5, 5))


scipy: 4.165651627465189


C:\Users\HYPERPC\AppData\Local\Temp\ipykernel_3936\602708007.py:21: RuntimeWarning: divide by zero encountered in log
  return 1/2 + 1/2 * erf((np.log(x) - 2)/(np.sqrt(0.4)))
C:\Users\HYPERPC\AppData\Local\Temp\ipykernel_3936\1631135038.py:16: RuntimeWarning: invalid value encountered in scalar multiply
  return mult * sum(generatedValues)
C:\Users\HYPERPC\AppData\Local\Temp\ipykernel_3936\602708007.py:21: RuntimeWarning: invalid value encountered in log
  return 1/2 + 1/2 * erf((np.log(x) - 2)/(np.sqrt(0.4)))
